<a href="https://colab.research.google.com/github/TerryYeh54147/20250420-PNLP/blob/main/Lesson_45_%E6%95%B4%E5%90%88_Elasticsearch%E3%80%81Ollama%E3%80%81Llama_3_2_%E5%8F%8A_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 45 整合Elasticsearch、Ollama、Llama 3.2 及 LangChain

此 Colab 建議使用 T4 GPU 15GB RAM。

### Table of Contents

* [步驟1：安裝與編譯套件](#packages)
* [步驟2：載入 PDF 檔案及分片資料](#create_folder)
* [步驟3：載入資料夾中檔案為並轉換為向量儲存](#load_file_and_to_vector)
* [步驟4：啟動 Ollama 提供地端大語言模型管理](#ollama)
* [步驟5：載入 Llama 3.2 1B 模型文字聊天](#ollama_chat)
* [步驟6：載入 Llama 3.2 11B 多模態圖像辨識](#ollama_image)
* [步驟7：載入 Llama 3.2 1B 模型回答 RAG 問題](#chat)
* [步驟8：比較相似度檢索和 Contextual Compression 檢索策略](#similarity)


In [ ]:
#@markdown <a id="packages" name="packages"></a>
#@markdown # **步驟 1：安裝套件** 🏗️
#@markdown 執行這個 Cell 會安裝所需要的套件
#@markdown ---
import locale,warnings

def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

from IPython.display import clear_output

warnings.filterwarnings("ignore", message="Convert_system_message_to_human will be deprecated!", category=UserWarning)

!apt-get install pciutils lshw

!pip install langchain -U langchain-community langchain-huggingface
!pip install langchain-elasticsearch
!pip install sentence-transformers
!pip install ollama
!pip install langchain-ollama
!pip install python-docx
!pip install PyMuPDF
!pip install faiss-cpu
!pip install pypdf==4.1.0
!pip install --upgrade --quiet "unstructured[pdf]" "unstructured[txt]"
!pip install Pillow

clear_output()
print("套件已安裝完成")

套件已安裝完成


In [ ]:
#@markdown <a id="create_folder" name="create_folder"></a>
#@markdown # **步驟 2：載入 PDF 檔案及分片資料** 🗂️
#@markdown 執行這個 Cell 之前，請將檔案放在此 Colab 的 ./data/ 資料夾中。
#@markdown ---
import os

doc_path = "./data/"
if not os.path.exists(doc_path):
    os.makedirs(doc_path)

!wget -O data/運動≠sports：本土運動觀念初探.pdf https://www.ios.sinica.edu.tw/people/personal/ctang/Se_C_04.pdf

clear_output()

print("再繼續執行下面的 cell 之前，請確認已經上傳了你要進行 RAG 的檔案至 Google Colab 的 ./data/ 資料夾中了。")

再繼續執行下面的 cell 之前，請確認已經上傳了你要進行 RAG 的檔案至 Google Colab 的 ./data/ 資料夾中了。


In [ ]:
#@markdown <a id="load_file_and_to_vector" name="load_file_and_to_vector"></a>
#@markdown # **步驟 3：載入資料夾中所有檔案為 LangChain Document 物件，並將之轉換為向量儲存** 🕑
#@markdown 執行這個 Cell 之前，請將檔案放在此 Colab 的 ./data/ 資料夾中。
#@markdown 建構 LangChain Agent，需要載入和預處理文件，使用 LangChain 的 DirectoryLoader 來載入資料夾中的PDF檔案，利用 RecursiveCharacterTextSplitter 將文字拆分成可管理的 Chunk (區塊)，拆分的手法決定了檢索是否精確。
#@markdown 執行此 cell 會將檔案載入，自動分片檔案，接著轉成向量，執行時間視你的檔案大小而不同，請靜待 cell 執行完成後再繼續。
#@markdown ```python
#@markdown ```+-data
#@markdown ```  +-你的檔案1.pdf
#@markdown ```  +-你的檔案2.pdf
#@markdown ```
#@markdown ---

import pandas as pd
from langchain_community.document_loaders import DirectoryLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import DataFrameLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

model_name = "sentence-transformers/distiluse-base-multilingual-cased-v1"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

from langchain.embeddings import HuggingFaceEmbeddings
from langchain_elasticsearch import ElasticsearchStore

directory = "./data"

loader = DirectoryLoader('./data', glob="**/*.*", show_progress=True)
docs = loader.load()

bIfElasticSearch = False
bIfVectorExists = False

if bIfElasticSearch:
  if not bIfVectorExists:
    def embedding_vectordb(docs):
        vectordb = ElasticsearchStore.from_documents(
            docs,
            index_name="documents_index",
            es_api_key="your_elasticsearch_api_key",
            embedding=embeddings,
            es_cloud_id="your_elasticsearch_cloud_id"
        )
        return vectordb

    vectordb = embedding_vectordb(docs)

  else:
    def embedding_vectordb():
        vectordb = ElasticsearchStore(
            embedding=embeddings,
            index_name="documents_index",
            es_api_key="your_elasticsearch_api_key",
            es_cloud_id="your_elasticsearch_cloud_id"
        )
        return vectordb

    vectordb = embedding_vectordb()

# Create DataFrame
df_extra = pd.DataFrame([(doc.page_content, doc.metadata) for doc in docs], columns=["page_content", "metadata"])
df_extra["type"] = df_extra["metadata"].apply(lambda x: x.get("type", ""))
df_extra['page_content'] = df_extra['page_content'].apply(lambda x: x.replace('\n', ' ') if isinstance(x, str) else x)
df_extra['source'] = df_extra['metadata'].apply(lambda x: x.get('source', ''))

# Remove metadata column
if "metadata" in df_extra.columns:
    del df_extra["metadata"]

# Load data from DataFrame
extra_loader = DataFrameLoader(df_extra, page_content_column="page_content")
extra_data = extra_loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
)
extra_documents = text_splitter.split_documents(extra_data)

vectordb = FAISS.from_documents(extra_documents, embeddings)

clear_output()

print("你的個人資料夾 ./data/　下的所有檔案，皆已轉成向量資料庫，請勿重複執行此 cell。")

你的個人資料夾 ./data/　下的所有檔案，皆已轉成向量資料庫，請勿重複執行此 cell。


本 Colab 使用 FAISS 作為向量資料庫，如要使用雲端的 ElasticSearch 服務，則在上 cell 中取代 index_name、es_api_key 和 es_cloud_id，如
建立 Elastic Cloud 帳戶、部署專案憑證，或者使用 Docker 後設定 Elasticsearch 來取得授權 key。Elastic Cloud，參考: https://cloud.elastic.co/login?redirectTo=%2Fhome

In [ ]:
#@markdown <a id="ollama" name="ollama"></a>
#@markdown # **步驟 4：啟動 Ollama 背景服務，以及 Llama 3.2 3B 模型** 🧬
#@markdown 執行這個 Cell 需要一點時間，請稍待片刻，不要重複執行。

#@markdown ---

import subprocess
import time
import requests

!curl https://ollama.ai/install.sh | sh

print("啟動 Ollama 服務...")
!nohup ollama serve &

server_up = False
server_url = "http://localhost:11434"

print("等待 Ollama 服務啟動...")
for _ in range(12):  # Check up to 12 times (approx. 1 minute)
    try:
        response = requests.get(server_url, timeout=5)
        if response.status_code == 200:
            server_up = True
            print("Ollama 服務已啟動。")
            break
    except requests.exceptions.RequestException:
        pass
    time.sleep(5)

if not server_up:
    print("Ollama 服務啟動失敗，請檢查伺服器設定。")
else:
    # Step 3: Pull the model
    print("下載模型中...")
    !ollama pull llama3.2:3b
    # !ollama pull gemma2
    # !ollama pull llama3.1:8b
    clear_output()
    print("模型已下載完成，Ollama 服務已啟動。")

# clear_output()
!ollama list

模型已下載


In [ ]:
#@markdown <a id="ollama_chat" name="ollama_chat"></a>
#@markdown # **步驟 5：載入 Llama 3.2 3B 模型文字聊天** 🔩
#@markdown 從 Ollama 載入 LLaMA 模型文字聊天。

#@markdown ---
from langchain_ollama import ChatOllama

def load_model(system_prompt):
    llm = ChatOllama(
        model="llama3.2:1b",
        temperature=0,
        system_prompt=system_prompt  # Add the system prompt here
    )
    return llm

def chat_with_model():
    system_prompt = "You are a helpful assistant that provides detailed and accurate answers to questions and always reply in Traditional Chinese NOT Simplified Chinese. And don't include `中国` in the response. Translate English response to Tranditional Chinese."
    llm = load_model(system_prompt)

    # Example of starting a chat
    print("歡迎來到聊天機器人! 請問有什麼問題? (輸入 'exit' 來結束對話)")
    while True:
        user_input = input("你: ")
        if user_input.lower() == 'exit':
            print("聊天結束，再見!")
            break

        # Use the stream method for streaming responses
        print("模型: ", end="")
        for response in llm.stream(user_input):  # Updated to stream responses
            print(response.content, end='', flush=True)  # Print in real-time without new line
        print()  # Move to next line after the response


chat_with_model()

歡迎來到聊天機器人! 請問有什麼問題? (輸入 'exit' 來結束對話)
你: 列出三樣去士林夜市必吃的小吃?
模型: 去士林夜市的美食之都，以下是三樣小吃你 shouldn'tmiss：

1. **士林夜市的烤肉**: 這裡的烤肉非常受歡迎，選擇不同的肉類和調味料可以找到您的口感喜好。
2. **士林夜市的海鲜**: 這裡的海鮮很豐富，包括生魚片、蟹肉、虾等。您可以選擇不同的海鮮和配料來創造您的 Own 美味。
3. **士林夜市的糖醬牛排**: 這是一個非常受歡迎的夜市小吃，牛排是由高質量的牛肉和精美的糖醬所組成。
你: exit
聊天結束，再見!


In [ ]:
#@markdown <a id="ollama_image" name="ollama_image"></a>
#@markdown # **步驟 6：載入 Llama 3.2 11B 多模態圖像辨識 (模型需要較大VRAM記憶體)** 🔩
#@markdown 從 Ollama 載入 LLaMA 模型以圖片聊天。

#@markdown ---

import requests
from PIL import Image
from io import BytesIO

# unsloth/Llama-3.2-11B-Vision-Instruct-bnb-4bit

# image_url = 'https://drive.google.com/uc?id=1z-4NFODGpZgrXOx1Btw2zV7CISp2xfQB&export=download'  # Replace with your image URL
# response = requests.get(image_url)
# image = Image.open(BytesIO(response.content))

In [ ]:
#@markdown <a id="chat" name="chat"></a>
#@markdown # **步驟 7：載入 Llama 3.2 1B 模型 RAG 回答問題** 🔩
#@markdown 從 Ollama 載入 LLaMA 模型以處理問答任務。LLaMA 模型根據檢索到的 PDF 執行 RAG 查詢並生成答案。
#@markdown ```python
#@markdown 台灣人究竟如何理解運動，與西方人對sport的理解又有何差異?
#@markdown ```
#@markdown ---

# import time
# def get_response(chain, query):
#     response = ""
#     start_time = time.time()
#     for s in chain.stream(query):
#       response += s.get("result", "")
#       print(s.get("result", ""), end="", flush=True)
#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print(f"\nTime taken to generate answer: {elapsed_time:.2f} seconds")
#     return response
# question = "台灣人究竟如何理解運動，與西方人對sport的理解又有何差異?"
# response = get_response(chain, question)

from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
import time

def load_model():
    llm = ChatOllama(
        model="llama3.2:1b",
        temperature=0,
    )
    return llm

llm = load_model()

def setup_prompt_chain(llm, vectordb):
    template = """You're PDF Agent, name Genie.
    {context}
    Please answer the question you have. Your answers should be concise and limited to a maximum of 512 words, ensuring that each response is a complete sentence. Avoid providing any extra out-of-context information, and always reply in Traditional Chinese (not Simplified Chinese). Additionally, do not include the term "中国" in your responses, and always reply in Traditional Chinese.
    Question: {question}
    Answer:"""

    prompt = PromptTemplate(template=template, input_variables=["context", "question"])
    chain_type_kwargs = {"prompt": prompt}
    retriever = vectordb.as_retriever(search_kwargs={"k": 3})

    chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs=chain_type_kwargs,
    )
    return chain

chain = setup_prompt_chain(llm, vectordb)

def get_response(chain, query):
    response = ""
    start_time = time.time()
    # Stream output from the chain
    for s in chain.stream(query):
        response += s.get("result", "")
        print(s.get("result", ""), end="", flush=True)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"\nTime taken to generate answer: {elapsed_time:.2f} seconds")
    return response

def chatbot(chain):
    print("歡迎來到聊天機器人! 請問有什麼問題? (輸入 'exit' 來結束對話)")
    while True:
        question = input("你: ")
        if question.lower() == 'exit':
            print("聊天結束，再見!")
            break
        response = get_response(chain, question)

chatbot(chain)


歡迎來到聊天機器人! 請問有什麼問題? (輸入 'exit' 來結束對話)
你: 台灣人究竟如何理解運動，與西方人對sport的理解又有何差異?
台湾人在对运动与西方人的理解方面存在一些不同之处。虽然许多台湾人也认可运动为一种重要的活动，但他们往往将其与西方人的 sport 的区分开来，尤其是在 sports 中的竞争性质上。

例如，台湾人通常不认为散步、泡温泉或三溫暖等活动是运动，而是视之为休闲活动。相反，许多西方人会将这些活动与运动联系起来，并认为它们具有竞争性的元素。

另一方面，台湾人也可能对 sports 的理解有所不同，因为他们往往强调运动的身体和心理健康方面。例如，他们可能会关注运动的效果、训练方法等，而不是仅仅关注竞争性质。

此外，台湾人在对 sports 的认知中还存在一些文化差异。例如，许多台湾人可能不会将 sports 与 competition联系起来，甚至可能认为 sports 是一种休闲活动。

综上所述，台湾人与西方人的运动理解方面有着不同的差异。虽然他们都认可运动为一种重要的活动，但他们往往将其与西方人的 sport 的区分开来，并且可能对 sports 的理解有所不同。
Time taken to generate answer: 3.62 seconds
你: exit
聊天結束，再見!


In [ ]:
#@markdown <a id="similarity" name="similarity"></a>
#@markdown # **步驟 8：比較相似度檢索和 Contextual Compression 檢索策略** 🎲
#@markdown
#@markdown 為了依據查詢檢索相關文件，可以使用向量資料庫中的 similarity_search 方法。此方法將搜尋與查詢語義相似的文件並傳回前 k 個相似的段落，接著再交由 RAG 查詢並生成答案。

#@markdown ---

question = "台灣人究竟如何理解運動，與西方人對sport的理解又有何差異?"

docs = vectordb.similarity_search(question,k=3)

# Check the length of the document
print("找到",len(docs), "個段落")

# Check the content of the first document
docs[0].page_content

找到 3 個段落


'關於台灣本土運動觀念的研究，迄今基本上還是一片空白。儘管如此，有  些研究或是觸及了運動與體育概念區分的問題，或是探討了華人傳統的身體文  化，並觸及像養生運動這樣的觀念，仍值得在此提出來討論。  台灣的學者其實很早便意識到傳統與現代身體活體間的斷裂，可惜受限於  體育學的訓練背景，對運動所蘊涵的現代性並不特別敏感。例如，台灣戰後體育  學界的先驅，曾留學德國，自大陸來台的江良規，在所著的教科書中開宗明義做  了如下的澄清：「干戈、技擊、雜技、百戲等項目，形式上雖與今日流行的各項  體育活動相類同，但本質上卻有顯著區別」。這些華人的傳統活動或是軍事訓練  或是康樂活動，不能與今日的體育相提並論。而之所以會出現體育這個新字彙，  在於「新的字彙必有其獨特含義表示一種新的認識和觀念，這種認識和觀念的完  整意義，無法借用舊有名詞來表達時，新名詞才應運而生，否則不必多此一舉」。  因此首先必須認清「體育這一種觀念，是舶來的新思想，用舊有眼光來加以衡量，  難免會造成盲人摸象的錯覺」。事實上，傳統的看法不曾認為蹴鞠、投壺、弄丸  等遊戲或身體活動具有教育的意義。因此，他主張「遊戲、體操、運動競技等各  式各樣的身體活動是固有文化的一部分，然體育卻是時代的產物，前人只知有遊  戲或運動，但不知有體育」（江良規，1968）。  雖然江氏在書中清楚標示出以「競技運動」來譯sport，但從他認為「前人  只知有遊戲或運動，但不知有體育」，以及認為運動競技是固有文化的一部分來  看，多少還是把sport與exercise混同，並未把sport當做核心的概念，賦予其特別  的注意 – 儘管他認為英人Archibald Maclaren（他誤認的體育一詞的發明者）之  7 此處的回顧以國內的文獻為主，對其他相關外文文獻的討論見湯志傑（2008）。  4  所以能超越歐陸體操的傳統，發展出體育的觀點，8是受到競技運動的影響，領  悟到身體活動的價值不限於強力健身醫療軍訓等片面目的，而應當向生活教育的  目的進軍。  繼江氏之後，許義雄（1973）也探討了「接受外來語的正確觀念」的問題，  並特別就「競技運動」的譯法斟酌，指出這個譯法源自日語，應是運動競技、田  徑賽的略語，9與sport雖有相互蘊涵的關係，但絕非等值。許氏進而藉考察sport'

結論：

本範例提供了FAISS向量資料庫，與ElasticSearch 的儲存/讀取方式，利用 Llama 3.2 1B的小模型來做 RAG，為了使用離線的 1B 模型，使用了 Ollama 套件來下載作為 API，至此我們沒有使用任何模型壓縮技術，而是使用了全精度的模型，佔用 T4 GPU 約 3GB 的VRAM，下圖為來自 Ollama 官方網站上對小模型在支援的語言，包括英語、德語、法語、義大利語、葡萄牙語、印地語、西班牙語和泰語。Llama 3.2 的訓練語言範圍比這 8 種官方支援語言更為廣泛。

<img src="https://ollama.com/assets/library/llama3.2/c1a51716-d8bb-4642-8044-48f5022b777d">